In [ ]:
# Install required libraries
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 pyngrok
!pip install Flask-CORS==3.0.10
# !pip uninstall flask itsdangerous werkzeug
!pip install flask==0.12.2 itsdangerous==0.24 Werkzeug==0.11.15
!pip install markupsafe==1.1.1
!pip install jinja2==2.11.3
!pip install flask_ngrok
!ngrok config add-authtoken 2bzRVZhoNmcvLkZq9o7r7MAOgic_6c9LEzt7GcGNWuHiXTEUJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,pipeline
)
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

In [ ]:
model_path="parthbelose/llama_2_7B_chat_fine_tuned_"

In [ ]:
model_path = "/content/drive/MyDrive/lamama"

In [ ]:
system_message="You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do not assume anything. "

In [ ]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = {"": 0}
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map=device_map
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
app = Flask(__name__)
CORS(app)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n"
@app.route('/', methods=['POST'])
def predict():
    try:
        global prompt
        data = request.get_json()
        message = data.get('message', '')
        prompt += f"{message} [/INST]"
        # print(prompt)
        predicted_message = pipe(prompt)
        prompt=predicted_message[0]['generated_text']
        predicted_message=predicted_message[0]['generated_text'].split("[/INST]")[-1]
        return jsonify({'predicted_message': predicted_message})

    except Exception as e:
        print('Error predicting message:', str(e))
        return jsonify({'error': 'Failed to predict message'}), 500

if __name__ == '__main__':
      # app.run(port=5000)
    public_url = ngrok.connect(addr="5001")
    print("Public URL:", public_url)

    app.run(port=5001)


Public URL: NgrokTunnel: "https://e680-35-194-181-27.ngrok-free.app" -> "http://localhost:5001"
[INST] <<SYS>>
You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do not assume anything. 
<</SYS>>

hi i fell unwell [/INST]


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[INST] <<SYS>>
You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do not assume anything. 
<</SYS>>

hi i fell unwell [/INST]  Hello there! 😊 Sorry to hear that you're feeling unwell. Can you tell me a bit more about how you're feeling? It's important to prioritize your health and well-being, and I'm here to help in any way I can. Please feel free to share any details or concerns you may have, and I'll do my best to provide helpful and safe responses. 🌟i feel lonely [/INST]
[INST] <<SYS>>
You are a helpful and joyous ment